In [13]:
import pandas as pd
df1=pd.read_csv('studentAssessment.csv')
df2=pd.read_csv('assessments.csv')

In [14]:
merged_df=df1.merge(df2,on=['id_assessment'],how='inner')


In [15]:
merged_df

,id_assessment,id_student,date_submitted,is_banked,score,code_module,code_presentation,assessment_type,date,weight
0,1752,11391,18,0,78.0,AAA,2013J,TMA,19.0,10.0
1,1752,28400,22,0,70.0,AAA,2013J,TMA,19.0,10.0
2,1752,31604,17,0,72.0,AAA,2013J,TMA,19.0,10.0
3,1752,32885,26,0,69.0,AAA,2013J,TMA,19.0,10.0
4,1752,38053,19,0,79.0,AAA,2013J,TMA,19.0,10.0
...,...,...,...,...,...,...,...,...,...,...
173907,37443,527538,227,0,60.0,GGG,2014J,CMA,229.0,0.0
173908,37443,534672,229,0,100.0,GGG,2014J,CMA,229.0,0.0
173909,37443,546286,215,0,80.0,GGG,2014J,CMA,229.0,0.0
173910,37443,546724,230,0,100.0,GGG,2014J,CMA,229.0,0.0


In [16]:
merged_df['id_student'].nunique()

23369

In [17]:
module_presentations = merged_df.groupby(['code_module', 'code_presentation'])
thresholds = module_presentations['date'].max().reset_index()
thresholds['LS20'] = thresholds['date'] * 0.2
thresholds['LS40'] = thresholds['date'] * 0.4
thresholds['LS60'] = thresholds['date'] * 0.6
thresholds['LS80'] = thresholds['date'] * 0.8
thresholds['LS100'] = thresholds['date']

In [18]:
merged_df = pd.merge(merged_df, thresholds, on=['code_module', 'code_presentation'])


In [19]:
# Define a function to check late submissions by thresholds
def calculate_late(row, thresholds):
    late_counts = {}
    for thresh in thresholds:
        late_counts[thresh] = int(row['date_submitted'] > row[thresh])
    return pd.Series(late_counts)

threshold_columns = ['LS20', 'LS40', 'LS60', 'LS80', 'LS100']
merged_df[threshold_columns] = merged_df.apply(lambda row: calculate_late(row, threshold_columns), axis=1)


In [20]:

# Summarize late submissions by student
late_summary = merged_df.groupby('id_student')[threshold_columns].sum().reset_index()

# Output results
print(late_summary)

       id_student  LS20  LS40  LS60  LS80  LS100
0            6516     4     3     2     1      0
1            8462     1     0     0     0      0
2           11391     4     3     2     1      0
3           23629     3     1     0     0      0
4           23698     7     6     5     3      2
...           ...   ...   ...   ...   ...    ...
23364     2698251     6     4     3     2      0
23365     2698257     4     3     2     1      0
23366     2698535     6     5     3     1      1
23367     2698577     4     3     2     1      0
23368     2698588     4     3     2     1      1

[23369 rows x 6 columns]


Student ID 6516:

Submitted 4 assessments late by 20% course completion.
Submitted only 3 late by 40% (improved timeliness).
By 100% completion, there were no late submissions.
Student ID 23698:

Submitted 7 assessments late by 20% completion.
Continued submitting late, with 2 late submissions still recorded by 100% completion.

In [9]:
# Example verification: Check the correct comparison of late submissions at each threshold for a specific student
student_id = 6516  # Example student ID
student_data = merged_df[merged_df['id_student'] == student_id]

# Verify late submission counts for LS20, LS40, LS60, LS80, LS100
for index, row in student_data.iterrows():
    print(f"Assessment ID {row['id_assessment']} - Submission Date: {row['date_submitted']}")
    print(f"Threshold Dates: LS20: {row['LS20']}, LS40: {row['LS40']}, LS60: {row['LS60']}, LS80: {row['LS80']}, LS100: {row['LS100']}")
    print(f"Late Submission Count: LS20 = {int(row['date_submitted'] > row['LS20'])}, LS40 = {int(row['date_submitted'] > row['LS40'])}, LS60 = {int(row['date_submitted'] > row['LS60'])}, LS80 = {int(row['date_submitted'] > row['LS80'])}, LS100 = {int(row['date_submitted'] > row['LS100'])}")
    print('---')


Assessment ID 1758 - Submission Date: 17
Threshold Dates: LS20: 0, LS40: 0, LS60: 0, LS80: 0, LS100: 0
Late Submission Count: LS20 = 1, LS40 = 1, LS60 = 1, LS80 = 1, LS100 = 1
---
Assessment ID 1759 - Submission Date: 51
Threshold Dates: LS20: 1, LS40: 0, LS60: 0, LS80: 0, LS100: 0
Late Submission Count: LS20 = 1, LS40 = 1, LS60 = 1, LS80 = 1, LS100 = 1
---
Assessment ID 1760 - Submission Date: 116
Threshold Dates: LS20: 1, LS40: 1, LS60: 0, LS80: 0, LS100: 0
Late Submission Count: LS20 = 1, LS40 = 1, LS60 = 1, LS80 = 1, LS100 = 1
---
Assessment ID 1761 - Submission Date: 164
Threshold Dates: LS20: 1, LS40: 1, LS60: 1, LS80: 0, LS100: 0
Late Submission Count: LS20 = 1, LS40 = 1, LS60 = 1, LS80 = 1, LS100 = 1
---
Assessment ID 1762 - Submission Date: 210
Threshold Dates: LS20: 1, LS40: 1, LS60: 1, LS80: 1, LS100: 0
Late Submission Count: LS20 = 1, LS40 = 1, LS60 = 1, LS80 = 1, LS100 = 1
---


In [21]:
late_summary1=late_summary.to_csv('late_summary1.csv')